# QA Deep Dive

In this QA Deep Dive notebook, we take a closer look at semantic ingestion, RAG, and retrieval, presenting the various customization options, and providing hints to help you make the most out of your QA application.

For getting started with basic QA usage, check out [QA Quick Start](./qa_quick_start.ipynb).

### Access required

The content of this notebook requires access to Deep Search capabilities which are not
available on the public access system.

[Contact us](https://ds4sd.github.io) if you are interested in exploring
these Deep Search capabilities.


### GenAI Integration required

When interacting with the virtual assistant, Deep Search requires a connection to a Generative AI API. Currently, we support connections to [watsonx.ai](https://www.ibm.com/products/watsonx-ai) or the IBM-internal GenAI platform BAM.

Deep Search allows custom GenAI configurations for each project.
In the following example you will require to work in a project which has such GenAI capabilities activated.

### Set notebook parameters


In [1]:
import os
from dotenv import load_dotenv
from pydantic import TypeAdapter

load_dotenv()

PROFILE_NAME = os.environ.get("DS_NB_PROFILE")  # profile to use; defaults to active one
PROJ_KEY = os.environ["DS_NB_PROJ_KEY"]  # project to use
INDEX_KEY = os.environ["DS_NB_QA_IDX_KEY"]
DOC_HASH = os.environ.get("DS_NB_QA_DOC_HASH")  # set only when targeting a specific doc
QUESTION = os.environ["DS_NB_QUESTION"]

# whether to ingest incrementally:
SKIP_INGESTED_DOCS = TypeAdapter(bool).validate_python(
    os.environ.get("DS_NB_SKIP_INGESTED_DOCS", True)
)
RETR_K = os.environ.get("DS_NB_RETR_K", 3)  # number of search results to retrieve
GEN_TIMEOUT = os.environ.get("DS_NB_GEN_TIMEOUT", 10)  # generation timeout in seconds
GEN_CTX_WIN_SIZE = os.environ.get("DS_NB_GEN_CTX_WIN_SIZE")  # max gen ctx window size

### Import example dependencies

In [2]:
import rich
from typing import Union
from IPython.display import display, Markdown
from deepsearch.cps.client.api import CpsApi
from deepsearch.cps.client.components.documents import create_private_data_source
from deepsearch.cps.client.components.elastic import (
    ElasticDataCollectionSource,
    ElasticProjectDataCollectionSource,
)
from deepsearch.cps.queries import RAGQuery, SemanticQuery
from deepsearch.cps.queries.results import RAGResult, SearchResult, SearchResultItem

### Connect to Deep Search

In [3]:
api = CpsApi.from_env(profile_name=PROFILE_NAME)

### Notebook utils

In [4]:
def render_provenance_url(
    api: CpsApi,
    coords: Union[ElasticDataCollectionSource, ElasticProjectDataCollectionSource],
    retr_item: SearchResultItem,
):
    ## compute URL to the document in the Deep Search UI
    item_index = int(retr_item.main_path[retr_item.main_path.rfind(".") + 1 :])
    doc_url = api.documents.generate_url(
        document_hash=retr_item.doc_hash,
        data_source=coords,
        item_index=item_index,
    )
    display(
        Markdown(
            f"The provenance of the answer can be inspected on the [source document]({doc_url})."
        )
    )

### Prepare data source

All semantic operations, i.e. ingestion, RAG, and retrieval, require a *data source* for defining the docs to operate on.

The cell below shows how to configure a *private* data source, i.e. for operating on a whole private collection or a given doc within one.
- set `document_hash` only when targeting a specific doc; when targeting the whole private collection instead, omit it or set it to `None`
- to use a *public* data source instead, switch to the commented code on the bottom of the cell

In [5]:
# configuring a private data source:
data_source = create_private_data_source(
    proj_key=PROJ_KEY,
    index_key=INDEX_KEY,
    document_hash=DOC_HASH,
)

# # configuring a public data source:
# from deepsearch.cps.client.components.documents import create_public_data_source
# data_source = create_public_data_source(
#     elastic_id="default",
#     index_key=INDEX_KEY,
#     document_hash=DOC_HASH,
# )

### Ingestion

If your data source has not yet been semantically indexed, you can ingest it into the vector DB as shown below. Otherwise you can skip this step.

Ingestion of already indexed docs is controlled via param `skip_ingested_docs`. 

Particularly when indexing whole collections, note that the larger the data source, the longer the ingestion duration.

In [6]:
task = api.documents.semantic_ingest(
    project=PROJ_KEY,
    data_source=data_source,
    skip_ingested_docs=SKIP_INGESTED_DOCS,
)

# wait for the ingestion task to finish
api.tasks.wait_for(PROJ_KEY, task.task_id)

/Users/pva/work/github.com/DS4SD/deepsearch-examples/.venv/lib/python3.10/site-packages/pydantic/main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `_LiteralGenericAlias` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{'ing_out': {}}

### RAG

Besides the standard RAG usage shown in [QA Quick Start](./qa_quick_start.ipynb), `RAGQuery` has numerous additional parameters for customizing aspects of retrieval, generation, and overall RAG pipeline:


In [7]:
help(RAGQuery)

Help on function RAGQuery in module deepsearch.cps.queries:

RAGQuery(question: str, *, project: Union[str, deepsearch.cps.client.components.projects.Project], data_source: Union[deepsearch.cps.client.components.documents.PrivateDataDocumentSource, deepsearch.cps.client.components.documents.PrivateDataCollectionSource, deepsearch.cps.client.components.documents.PublicDataDocumentSource], retr_k: int = 10, rerank: bool = False, text_weight: typing.Annotated[float, FieldInfo(default=PydanticUndefined, ge=0.0, le=1.0, multiple_of=0.1, extra={'strict': True})] = 0.1, model_id: Optional[str] = None, prompt_template: Optional[str] = None, gen_params: Optional[Dict[str, Any]] = None, gen_ctx_extr_method: Literal['window', 'page'] = 'window', gen_ctx_window_size: int = 5000, gen_ctx_window_lead_weight: float = 0.5, return_prompt: bool = False, chunk_refs: Optional[List[deepsearch.cps.queries.results.ChunkRef]] = None, gen_timeout: Optional[float] = None) -> deepsearch.cps.client.queries.query.

For instance, below we set the `return_prompt` parameter to get the actual instantiated prompt.

In [8]:
question_query = RAGQuery(
    question=QUESTION,
    project=PROJ_KEY,
    data_source=data_source,
    retr_k=RETR_K,
    return_prompt=True,
    gen_timeout=GEN_TIMEOUT,
    gen_ctx_window_size=GEN_CTX_WIN_SIZE,
)
api_output = api.queries.run(question_query)
rag_result = RAGResult.from_api_output(api_output)

rich.print(QUESTION)
rich.print(rag_result)

Where was the first European IBM lab located?

RAGResult(
    answers=[
        RAGAnswerItem(
            answer='The first European IBM laboratory was located in Adliswil, Switzerland, near Zurich.',
            grounding=RAGGroundingInfo(
                retr_items=[
                    SearchResultItem(
                        doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
                        main_path='main-text.71',
                        path_group=['main-text.69', 'main-text.71'],
                        chunk='Switzerland\nIn 1956, IBM opened their first European research laboratory in 
Adliswil, Switzerland, near Zurich. The lab moved to its own campus in neighboring Rüschlikon in 1962. The Zurich 
lab is staffed by a multicultural and interdisciplinary team of a few hundred permanent research staff members, 
graduate students and post-doctoral fellows, representing about 45 nationalities. Collocated with the lab is a 
Client Center (formerly the Industry Solutions Lab), an executive briefing facility demonstrating technology 
prototypes and solutions.',
                        source_is_text=True
                    )
                ],
                gen_ctx_paths=['main-text.71']
            ),
            prompt='Context information is below.\n---------------------\nIn 1956, IBM opened their first European 
research laboratory in Adliswil, Switzerland, near Zurich. The lab moved to its own campus in neighboring 
Rüschlikon in 1962. The Zurich lab is staffed by a multicultural and interdisciplinary team of a few hundred 
permanent research staff members, graduate students and post-doctoral fellows, representing about 45 nationalities.
Collocated with the lab is a Client Center (formerly the Industry Solutions Lab), an executive briefing facility 
demonstrating technology prototypes and solutions.\n---------------------\nGiven the context information and not 
prior knowledge, answer the query.\nQuery: Where was the first European IBM lab located?\nAnswer:\n'
        )
    ],
    search_result_items=[
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            main_path='main-text.71',
            path_group=['main-text.69', 'main-text.71'],
            chunk='Switzerland\nIn 1956, IBM opened their first European research laboratory in Adliswil, 
Switzerland, near Zurich. The lab moved to its own campus in neighboring Rüschlikon in 1962. The Zurich lab is 
staffed by a multicultural and interdisciplinary team of a few hundred permanent research staff members, graduate 
students and post-doctoral fellows, representing about 45 nationalities. Collocated with the lab is a Client Center
(formerly the Industry Solutions Lab), an executive briefing facility demonstrating technology prototypes and 
solutions.',
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            main_path='main-text.70',
            path_group=['main-text.69', 'main-text.70'],
            chunk='Switzerland\nIBM Research-Zurich (previously called IBM Zurich Research Laboratory, ZRL) is the 
European branch of IBM Research. It was opened in 1956 and is located in Rüschlikon, near Zurich, Switzerland.',
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            main_path='main-text.8',
            path_group=['main-text.7', 'main-text.8'],
            chunk="History\nThe roots of today's IBM Research began with the 1945 opening of the Watson Scientific 
Computing Laboratory at Columbia University. $^{[4]}$ This was the first IBM laboratory devoted to pure science and
later expanded into additional IBM Research locations in Westchester County, New York, starting in the 1950s, 
$^{[5][6]}$ including the Thomas J. Watson Research Center in 1961. [5][6]",
            source_is_text=

Additionally, we can generate a provenance URL to the document in the Deep Search UI:

In [9]:
render_provenance_url(
    api=api,
    coords=data_source.source,
    retr_item=rag_result.answers[0].grounding.retr_items[0],
)

The provenance of the answer can be inspected on the [source document](https://sds.app.accelerate.science/projects/b09ae7561a01dc7c4b0fd21a43bfd93d140766d1/library/private/6b70072911ad2794a3844dd44d1705a5ba37ca0b?search=JTdCJTIycHJpdmF0ZUNvbGxlY3Rpb24lMjIlM0ElMjI2YjcwMDcyOTExYWQyNzk0YTM4NDRkZDQ0ZDE3MDVhNWJhMzdjYTBiJTIyJTJDJTIydHlwZSUyMiUzQSUyMkRvY3VtZW50JTIyJTJDJTIyZXhwcmVzc2lvbiUyMiUzQSUyMmZpbGUtaW5mby5kb2N1bWVudC1oYXNoJTNBJTIwJTVDJTIyYjMwYmM2NjdhMzI0YWUxMTFkMDI1NTI2NTYzYjY3NGE4ZDNmZDg2OWJjMDdjOGZkMjA0YWE5NWIwNWQ0MWYwYyU1QyUyMiUyMiUyQyUyMmZpbHRlcnMlMjIlM0ElNUIlNUQlMkMlMjJzZWxlY3QlMjIlM0ElNUIlMjJfbmFtZSUyMiUyQyUyMmRlc2NyaXB0aW9uLmNvbGxlY3Rpb24lMjIlMkMlMjJwcm92JTIyJTJDJTIyZGVzY3JpcHRpb24udGl0bGUlMjIlMkMlMjJkZXNjcmlwdGlvbi5wdWJsaWNhdGlvbl9kYXRlJTIyJTJDJTIyZGVzY3JpcHRpb24udXJsX3JlZnMlMjIlNUQlMkMlMjJpdGVtSW5kZXglMjIlM0EwJTJDJTIycGFnZVNpemUlMjIlM0ExMCUyQyUyMnNlYXJjaEFmdGVySGlzdG9yeSUyMiUzQSU1QiU1RCUyQyUyMnZpZXdUeXBlJTIyJTNBJTIyc25pcHBldHMlMjIlMkMlMjJyZWNvcmRTZWxlY3Rpb24lMjIlM0ElN0IlMjJyZWNvcmQlMjIlM0ElN0IlMjJpZCUyMiUzQSUyMmIzMGJjNjY3YTMyNGFlMTExZDAyNTUyNjU2M2I2NzRhOGQzZmQ4NjliYzA3YzhmZDIwNGFhOTViMDVkNDFmMGMlMjIlN0QlMkMlMjJpdGVtSW5kZXglMjIlM0E3MSU3RCU3RA%3D%3D).

As shown below, we can also inspect the timing information of the query execution, including the time spent on each step of the pipeline:

In [10]:
rich.print(api_output.timings)

QueryTimings(
    overall=1.1124043259769678,
    tasks={
        'QA': TaskTimings(
            overall=1.112084336578846,
            details={
                'pipeline': 0.7129936516284943,
                'encode': 0.025656767189502716,
                'search': 0.01602264866232872,
                'rerank': 0.0,
                'extr_gen_ctx': 0.037559472024440765,
                'generate': 0.6326766759157181
            }
        )
    }
)

### Generating next-best answers

To generate next-best answers, we can make follow-up queries with respective `chunk_refs` based on a reference RAG result.

For example, below we use the previous RAG result as reference and generate answers based on its second-best and third-best items.

In [11]:
from typing import List

from deepsearch.cps.queries.results import RAGAnswerItem

reference_rag_result = rag_result

answer_items: List[RAGAnswerItem] = []
for i in range(1, 3):

    chunk_refs = [reference_rag_result.search_result_items[i]]

    question_query = RAGQuery(
        question=QUESTION,
        project=PROJ_KEY,
        data_source=data_source,
        retr_k=RETR_K,
        return_prompt=True,
        chunk_refs=chunk_refs,  # <==
        gen_ctx_window_size=GEN_CTX_WIN_SIZE,
        gen_timeout=GEN_TIMEOUT,
    )
    api_output = api.queries.run(question_query)
    new_rag_result = RAGResult.from_api_output(api_output)
    answer_items.append(new_rag_result.answers[0])

Here we print the resulting next-best answers in a table:

In [12]:
from rich.console import Console
from rich.table import Table

table = Table(title="Next-best answers", show_lines=True, width=100)

table.add_column("Prompt", style="yellow")
table.add_column("Answer", style="green")

for answer_item in answer_items:
    table.add_row(answer_item.prompt, answer_item.answer)

console = Console()
console.print(table)

                                         Next-best answers                                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                          ┃ Answer                                         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Context information is below.                   │ The first European IBM lab was located in      │
│ ---------------------                           │ Zurich, Switzerland, and opened in 1956. It is │
│ IBM Research-Zurich (previously called IBM      │ currently known as IBM Research-Zurich.        │
│ Zurich Research Laboratory, ZRL) is the         │                                                │
│ European branch of IBM Research. It was opened  │                                                │
│ in 1956 and is located in Rüschlikon, near      │                                                │
│ Zurich, Switzerland.                            │                                                │
│ ---------------------                           │                                                │
│ Given the context information and not prior     │                                                │
│ knowledge, answer the query.                    │                                                │
│ Query: Where was the first European IBM lab     │                                                │
│ located?                                        │                                                │
│ Answer:                                         │                                                │
│                                                 │                                                │
├─────────────────────────────────────────────────┼────────────────────────────────────────────────┤
│ Context information is below.                   │ The information does not directly state where  │
│ ---------------------                           │ the first European IBM lab was located; it     │
│ The roots of today's IBM Research began with    │ only mentions the Watson Scientific Computing  │
│ the 1945 opening of the Watson Scientific       │ Laboratory at Columbia University as the first │
│ Computing Laboratory at Columbia University.    │ IBM laboratory devoted to pure science.        │
│ $^{[4]}$ This was the first IBM laboratory      │ However, it does mention the expansion of IBM  │
│ devoted to pure science and later expanded into │ Research into locations in Westchester County, │
│ additional IBM Research locations in            │ New York, in the 1950s. To find out where the  │
│ Westchester County, New York, starting in the   │ first European IBM lab was located, one would  │
│ 1950s, $^{[5][6]}$ including the Thomas J.      │ need additional information.                   │
│ Watson Research Center in 1961. [5][6]          │                                                │
│ ---------------------                           │                                                │
│ Given the context information and not prior     │                                                │
│ knowledge, answer the query.                    │                                                │
│ Query: Where was the first European IBM lab     │                                                │
│ located?                                        │                                                │
│ Answer:                                         │                                                │
│                                                 │                                                │
└─────────────────────────────────────────────────┴────────────────────────────────────────────────┘

### Semantic retrieval

Besides the standard semantic retrieval usage shown in [QA Quick Start](./qa_quick_start.ipynb), `SemanticQuery` has numerous additional parameters:

In [13]:
help(SemanticQuery)

Help on function SemanticQuery in module deepsearch.cps.queries:

SemanticQuery(question: str, *, project: Union[str, deepsearch.cps.client.components.projects.Project], data_source: Union[deepsearch.cps.client.components.documents.PrivateDataDocumentSource, deepsearch.cps.client.components.documents.PrivateDataCollectionSource, deepsearch.cps.client.components.documents.PublicDataDocumentSource], retr_k: int = 10, rerank: bool = False, text_weight: typing.Annotated[float, FieldInfo(default=PydanticUndefined, ge=0.0, le=1.0, multiple_of=0.1, extra={'strict': True})] = 0.1) -> deepsearch.cps.client.queries.query.Query
    Create a semantic retrieval query
    
    Args:
        question (str): the natural-language query
        document_hash (str): hash of target document
        project (Union[str, Project]): project to use
        data_source (DataSource): the data source to query
        retr_k (int, optional): num of items to retrieve; defaults to 10
        rerank (bool, optional):

For instance, below we set:
- `rerank` to `True` to rerank the retrieved chunks, and
- `text_weight` to `0.8` to favor the lexical component of hybrid search

In [14]:
# submit natural-language query on document
question_query = SemanticQuery(
    question=QUESTION,
    project=PROJ_KEY,
    data_source=data_source,
    retr_k=RETR_K,
    rerank=True,
    text_weight=0.8,
)
api_output = api.queries.run(question_query)
rag_result = SearchResult.from_api_output(api_output)

rich.print(QUESTION)
rich.print(rag_result)

Where was the first European IBM lab located?

SearchResult(
    search_result_items=[
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            main_path='main-text.71',
            path_group=['main-text.69', 'main-text.71'],
            chunk='Switzerland\nIn 1956, IBM opened their first European research laboratory in Adliswil, 
Switzerland, near Zurich. The lab moved to its own campus in neighboring Rüschlikon in 1962. The Zurich lab is 
staffed by a multicultural and interdisciplinary team of a few hundred permanent research staff members, graduate 
students and post-doctoral fellows, representing about 45 nationalities. Collocated with the lab is a Client Center
(formerly the Industry Solutions Lab), an executive briefing facility demonstrating technology prototypes and 
solutions.',
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='234bc5cf2c860d49574b0ff7191c354b7bbc11472a09972c8b6108c62375ffc3',
            main_path='main-text.86',
            path_group=['main-text.85', 'main-text.86'],
            chunk="Research\nResearch has been part of IBM since its founding, and its organized efforts trace 
their roots back to 1945, when the Watson Scientific Computing Laboratory was founded at Columbia University in New
York City, converting a renovated fraternity house on Manhattan's West Side into IBM's first laboratory. Now, IBM 
Research constitutes the largest industrial research organization in the world, with 12 labs on 6 continents. [161]
IBM Research is headquartered at the Thomas J. Watson Research Center in New York, and facilities include the 
Almaden lab in California, Austin lab in Texas, Australia lab in Melbourne, Brazil lab in São Paulo and Rio de 
Janeiro, China lab in Beijing and Shanghai, Ireland lab in Dublin, Haifa lab in Israel, India lab in Delhi and 
Bangalore, Tokyo lab, Zurichlab and Africa lab in Nairobi.",
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            main_path='main-text.70',
            path_group=['main-text.69', 'main-text.70'],
            chunk='Switzerland\nIBM Research-Zurich (previously called IBM Zurich Research Laboratory, ZRL) is the 
European branch of IBM Research. It was opened in 1956 and is located in Rüschlikon, near Zurich, Switzerland.',
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            main_path='main-text.62',
            path_group=['main-text.61', 'main-text.62'],
            chunk="Israel\nIBM Research-Haifa, previously known as the Haifa Research Lab (HRL) was founded as a 
small scientific center in 1972. $^{[55]}$ Since then, it has grown into a major lab that leads the development of 
innovative technologies and solutions for the IBM corporation. The lab's offices are situated in three locations 
across Israel: Haifa, Tel Aviv, and Beer Sheva.",
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            main_path='main-text.44',
            path_group=['main-text.43', 'main-text.44'],
            chunk="Australia\nIBM Research-Australia is a research and development laboratory established by IBM 
Research in 2009 in Melbourne. [39] It is involved in social media, interactive content, healthcare analytics and 
services research, multimedia analytics, and genomics. The lab is headed by Vice President and Lab Director Joanna 
Batstone. [40] It was to be the company's first laboratory combining research and development in a single 
organisation. [41]",
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='fa7ce2f66a7a5e061813d36348425f81d9e7ebc23454d8e63359fb5e28ab8dd0',
            main_path='main-text.1',
            path_g

Again here, we can see the time spent on each step of the pipeline:

In [15]:
rich.print(api_output.timings)

QueryTimings(
    overall=2.106315772049129,
    tasks={
        'QA': TaskTimings(
            overall=2.1059834593907,
            details={
                'pipeline': 1.7860895283520222,
                'encode': 0.0273594930768013,
                'search': 0.017811238765716553,
                'rerank': 1.739860225468874
            }
        )
    }
)